In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from gekko import GEKKO

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("MPC-V3.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel, DP
from Logic import logic_bat, logic_actions
from Funcs_Logic_DP import (get_price, get_emissions, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)
from MPC import MPC

In [4]:
df = merge('h16')
M_h16 = MPC()
M_h16.MPCopt(df)

apm 192.38.81.6_gk_model1 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          217
   Intermediates:            0
   Connections  :            0
   Equations    :          170
   Residuals    :          170
 
 Number of state variables:            217
 Number of total equations: -          169
 Number of slack variables: -           48
 ---------------------------------------
 Degrees of freedom       :              0
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program conta

 159  7.8669325e+01 5.54e-11 1.20e-01  -7.0 1.20e-01    -  1.00e+00 1.00e+00H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160  7.8669325e+01 1.21e-12 1.99e-05  -7.0 8.63e-06  -1.4 1.00e+00 1.00e+00h  1
 161  7.8669325e+01 2.85e-12 7.31e-01  -7.0 4.03e-02    -  1.00e+00 3.12e-02h  6
 162  7.8669325e+01 5.40e-14 1.10e-06  -7.0 9.16e-06  -1.9 1.00e+00 1.00e+00h  1
 163  7.8669325e+01 1.06e-11 6.95e-01  -7.0 3.98e-02    -  1.00e+00 6.25e-02h  5
 164  7.8669325e+01 4.12e-13 1.38e-06  -7.0 4.84e-05  -2.4 1.00e+00 1.00e+00h  1
 165  7.8669325e+01 4.84e-11 8.22e-02  -7.0 3.98e-02    -  1.00e+00 1.00e+00H  1
 166  7.8669325e+01 5.61e-12 1.77e-04  -7.0 8.95e-05  -2.8 1.00e+00 1.00e+00h  1
 167  7.8669325e+01 1.37e-12 1.13e+00  -7.0 6.23e-02    -  1.00e+00 3.12e-02h  6
 168  7.8669325e+01 1.72e-14 9.69e-07  -7.0 9.08e-06  -2.4 1.00e+00 1.00e+00h  1

Number of Iterations....: 168

                                   (scaled)                 (unscaled)
Object

0     0.0
1    -0.0
2    -0.0
3     0.0
4     0.0
5     3.5
6    -3.5
7     0.0
8     7.0
9    -3.2
10    1.2
11   -5.0
12    0.0
13   -0.0
14    1.1
15    0.4
16    1.9
17    7.0
18    2.6
19   -0.0
20   -1.5
21   -4.5
22   -7.0
23   -0.0
Name: charge, dtype: float64